In [6]:
from equations import EquationSolver as EQ
import sympy as symp
import numpy as np
from ambiance import Atmosphere

In [2]:
def weight_calcs(W_max, We_Wmax,
                 num_passengers = 1200,
                 passenger_weight = 205,
                 
                ):
    W_empty = We_Wmax * W_max
    W_payload = passenger_weight * num_passengers
    W_fuel = W_max - W_payload - W_empty
    return W_empty, W_payload, W_fuel    

In [8]:
wing_params = EQ("AR - b**2 / S")
wing_params.solve_for('S', b=311,AR=9.5)

In [31]:
def find_poly_coeffs(weight,rho,C_D0,S,T_A0,e,AR):
    A = rho*C_D0*S / 2.
    B = -T_A0
    C = 2 * weight **2. / rho / S / e / AR / np.pi
    return A,B,C


def V0max_at_sea(weight,rho,C_D0,S,T_A0,e,AR):
    A,B,C = find_poly_coeffs(weight,rho,C_D0,S,T_A0,e,AR)
    roots = max(np.roots([A,B,C]))
    V0 = roots**.5    
    return V0

def RCmax(weight,rho,C_D0,S,T_A0,e,AR):
    A,B,C = find_poly_coeffs(weight,rho,C_D0,S,T_A0,e,AR)
    roots = max(np.roots([3*A,B,-C]))
    V0 = roots**.5
    RC_max = (A*V0**3 + B * V0 + C/V0) / (-weight)
    return RC_max

def max_ceil_sigma(weight,rho,C_D0,S,T_A0,e,AR):
    A,B,C = find_poly_coeffs(weight,rho,C_D0,S,T_A0,e,AR)
    sigma_max = np.sqrt(4*A*C/B**2)
    return sigma_max

np.float64(1428.7203941862645)

In [35]:
def Cl(W,rho,V,S):
    return 2*W / rho / V**2 / S
    
def Cd(C_D0,W,rho,V,S,e,AR):
    return C_D0 + Cl(W,rho,V,S)**2 / np.pi / e /AR
    
def cruise_range(TSFC,rho,V,S,C_D0,e,AR,W_init,W_final):
    Cl_init = Cl(W_init,rho,V,S)
    Cl_final = Cl(W_final,rho,V,S)
    Cd_init = Cd(C_D0,W_init,rho,V,S,e,AR)
    Cd_final = Cd(C_D0,W_final,rho,V,S,e,AR)

    Cl_Cd_ratio = 0.5 * (np.sqrt(Cl_init)/Cd_init + np.sqrt(Cl_final)/Cd_final)
    
    cruise_range = (2/TSFC) *\
    sp.sqrt(2 / (rho*S)) *\
    Cl_Cd_ratio *\
    (sp.sqrt(Wi)-sp.sqrt(Wf))
    return cruise_range

In [ ]:
def V_stall(weight, rho, S, Clmax):
    return np.sqrt(
        2*weight / rho / S / Clmax
    )

def takeoff_distance(weight, S, C_D0, rho, Clmax, Ts,
                    h_obstacle = 35):
    V_1 = 1.2 * V_stall(weight, rho, S, Clmax)
    D_1 = C_D0*rho**(V_1**2)*S/2
    F_1 = (Ts - D_1)
    F_s = (Ts - mu*W)
    S_1 = W/2/g*(V_1**2)/(F_s - F_1)*np.log(F_s/F_1)
    A,_,C = find_poly_coeffs(weight,rho,C_D0,S,T_A0,e,AR):
    P_R = A * V_1**3 + C / V_1
    P_A = V_1 * Ts
    RC = (P_A - P_R) / weight
    S_2 = h_obstacle / RC * V_1
    takeoff_distance = S_1 + S_2